<a href="https://colab.research.google.com/github/dylanjcastillo/nlp-tweets-classification/blob/main/notebooks/models/2_BETO_model_with_data_labeled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --quiet emoji transformers

## Imports and settings

In [3]:
import gc
import joblib
import json
import os
import random
import re
import warnings

import torch

import pandas as pd
import numpy as np

from emoji import demojize
from tqdm import tqdm
from torch import nn

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

from scipy.special import softmax
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, \
                            cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder


DATA_PATH = "/content/drive/MyDrive/nlp-tweets-classification/"
SEED = 42

random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

warnings.filterwarnings('ignore')

## Utility functions

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
          return len(self.labels)


class InferenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings["input_ids"])
    
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}        


class MyTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # You pass the class weights when instantiating the Trainer
        self.class_weights = torch.as_tensor(class_weights, dtype=torch.float).cuda()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    precision_ind, recall_ind, f1_ind, _ = precision_recall_fscore_support(labels, preds, average=None)
    kappa = cohen_kappa_score(labels, preds)
    return {
        'kappa': kappa,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'l1_f1': f1_ind[0],
        'l2_f1': f1_ind[1],
        'l3_f1': f1_ind[2],
        'l1_precision': precision_ind[0],
        'l2_precision': precision_ind[1],
        'l3_precision': precision_ind[2],
        'l1_recall': recall_ind[0],
        'l2_recall': recall_ind[1],
        'l3_recall': recall_ind[2],
    }

def get_mode_rows(a):
    a = np.ascontiguousarray(a)
    void_dt = np.dtype((np.void, a.dtype.itemsize * np.prod(a.shape[1:])))
    _, ids, count = np.unique(a.view(void_dt).ravel(), return_index=1, return_counts=1)
    largest_count_id = ids[count.argmax()]
    most_frequent_row = a[largest_count_id]
    return most_frequent_row

def compute_class_weights(y): 
  classes = np.sort(np.unique(y))
  class_weights = [1 for c in classes] 
  if len(classes) > 1:
    for i, v in enumerate(classes):
      class_weights[i] = 1 - ((y == v).sum() / y.shape[0])
  return class_weights

def clean_text(text):
    tokens = text.split()
    tokens = [t for t in tokens if not 'http' in t]
    tokens = [demojize(t, language='es') for t in tokens]
    tokens = [t.replace('@', '') for t in tokens]
    return " ".join(tokens)

def read_data(col_target, use_full_dataset=False):
  df = pd.read_json(DATA_PATH + "all_multiclass_20220911.json").dropna(subset=[col_target])
  if not use_full_dataset:
    df_sample = pd.read_json(DATA_PATH + "tweets_sample.json")
    df_test = df_sample[~df_sample.id.isin(df.id)].reset_index()
  else:
    if col_target == "target_layer_1":
      df_test = (
          pd.read_csv(DATA_PATH + "tweets_traducidos.csv")
      )
      df_test["text"] = df_test.texto_traducido.combine_first(df_test.text)
    elif col_target == "target_layer_2":
      df_test = (
          pd.read_csv(DATA_PATH + "output_target_layer_1.csv")
          .query("target_layer_1 == 'transparencia'")
          .reset_index()
      )
    elif col_target == "target_layer_3":
      df_test = (
          pd.read_csv(DATA_PATH + "output_target_layer_2.csv")
          .query("target_layer_2 == 'decisiones'")
          .reset_index()
      )

  df["clean_text"] = df.text.map(clean_text)
  df_test["clean_text"] = df_test.text.map(clean_text)
  print(df[col_target].value_counts())
  return df, df_test

def list_arrays_to_matrix(list_arrays):
  shape = list(list_arrays[0].shape)
  shape[:0] = [len(list_arrays)]
  return np.concatenate(list_arrays).reshape(shape)

In [5]:
col_target = "target_layer_2"
model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
n_splits = 10 
use_full_dataset = True 

## Train model

In [ ]:
df, df_test = read_data(col_target, use_full_dataset=use_full_dataset)

gc.collect()
torch.cuda.empty_cache()

df_results = pd.DataFrame()
df_label = df[["clean_text", col_target]]
df_label = df_label.sample(frac=1).reset_index(drop=True)
df_label["kfold"] = -1

if n_splits > 1:
  kf = StratifiedKFold(n_splits=n_splits)
  
  for f, (t, v) in enumerate(kf.split(X=df_label, y=df_label[col_target])):
      df_label.loc[v, "kfold"] = f
else:
  ss = StratifiedShuffleSplit(n_splits=1, test_size=0.15)
  f = next(ss.split(X=df_label, y=df_label[col_target]))
  t = f[0]
  v = f[1]
  df_label.loc[v, "kfold"] = 0

f1_scores = []
l1_f1_scores = [] 
l2_f1_scores = [] 
l3_f1_scores = [] 

precision_scores = []
l1_precision_scores = []
l2_precision_scores = []
l3_precision_scores = []

recall_scores = []
l1_recall_scores = []
l2_recall_scores = []
l3_recall_scores = []

acc_scores = []
test_probas = []
kappa_scores = []

le = LabelEncoder()

print(f"Training {model_checkpoint}")
for f in range(n_splits):
    df_train = df_label[df_label.kfold != f].reset_index(drop=True)
    df_val = df_label[df_label.kfold == f].reset_index(drop=True)
    
    print(f"**** Split {f}: {df_train.shape[0]} observations for training / {df_val.shape[0]} observations for validation ****")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=False)

    train_encodings = tokenizer(df_train.clean_text.tolist(), truncation=True, padding=True, max_length=320)
    val_encodings = tokenizer(df_val.clean_text.tolist(), truncation=True, padding=True, max_length=320)
    test_encodings = tokenizer(df_test.clean_text.tolist(), truncation=True, padding=True, max_length=320)

    if not hasattr(le, "classes_"):
      le.fit(df_train[col_target])
      joblib.dump(le, DATA_PATH + f"models/{col_target}/label_encoder.joblib")

      def get_counts(arr):
        labels_dict = le.inverse_transform([0, 1, 2])
        return {
            "prob_" + labels_dict[0]: np.count_nonzero(arr == 0) / len(arr),
            "prob_" + labels_dict[1]: np.count_nonzero(arr == 1) / len(arr),
            "prob_" + labels_dict[2]: np.count_nonzero(arr == 2) / len(arr),
        }

    train_labels = le.transform(df_train[col_target])
    val_labels = le.transform(df_val[col_target])
    
    train_dataset = Dataset(train_encodings, train_labels)
    val_dataset = Dataset(val_encodings, val_labels)
    test_dataset = InferenceDataset(test_encodings)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=6,
        per_device_train_batch_size=64,   
        per_device_eval_batch_size=16, 
        warmup_steps=500,
        logging_dir='./logs',
        logging_steps=32,
        fp16=True,
        evaluation_strategy="epoch",
        save_strategy="no",
        metric_for_best_model="f1",
        learning_rate=4e-5,
    )

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(set(train_labels)))

    trainer = MyTrainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,             
        compute_metrics=compute_metrics,
        class_weights=compute_class_weights(le.transform(df[col_target]))
    )

    trainer.train()
    val_scores = trainer.evaluate()
    trainer.save_model(DATA_PATH + f"models/{col_target}/beto_fold_{f}")

    predictions = trainer.predict(test_dataset).predictions
    probabilities = softmax(predictions, axis=1)
    test_probas.append(probabilities)

    f1_scores.append(val_scores["eval_f1"])
    kappa_scores.append(val_scores["eval_kappa"])
    precision_scores.append(val_scores["eval_precision"])
    recall_scores.append(val_scores["eval_recall"])

    l1_f1_scores.append(val_scores["eval_l1_f1"])
    l2_f1_scores.append(val_scores["eval_l2_f1"])
    l3_f1_scores.append(val_scores["eval_l3_f1"])

    l1_precision_scores.append(val_scores["eval_l1_precision"])
    l2_precision_scores.append(val_scores["eval_l2_precision"])
    l3_precision_scores.append(val_scores["eval_l3_precision"])
    
    l1_recall_scores.append(val_scores["eval_l1_recall"])
    l2_recall_scores.append(val_scores["eval_l2_recall"])
    l3_recall_scores.append(val_scores["eval_l3_recall"])
    
df_results = pd.concat(
    [df_results, 
      pd.DataFrame(
          dict(
              target=col_target,
              model=[model_checkpoint],
              f1=np.mean(f1_scores),
              kappa=np.mean(kappa_scores),
              precision=np.mean(precision_scores),
              recall=np.mean(recall_scores),
              l1_f1=np.mean(l1_f1_scores),
              l2_f1=np.mean(l2_f1_scores),
              l3_f1=np.mean(l3_f1_scores),
              l1_precision=np.mean(l1_precision_scores),
              l2_precision=np.mean(l2_precision_scores),
              l3_precision=np.mean(l3_precision_scores),
              l1_recall=np.mean(l1_recall_scores),
              l2_recall=np.mean(l2_recall_scores),
              l3_recall=np.mean(l3_recall_scores),
              f1_scores=str([round(s, 2) for s in f1_scores]),
              kappa_scores=str([round(s, 2) for s in kappa_scores]),
              precision_scores=str([round(s, 2) for s in precision_scores]),
              recall_scores=str([round(s, 2) for s in recall_scores]),
          )
        )
]) 

df_results = df_results.rename(columns={
    "l1_f1": f"{le.classes_[0]}_f1",
    "l2_f1": f"{le.classes_[1]}_f1",
    "l3_f1": f"{le.classes_[2]}_f1",
    "l1_precision": f"{le.classes_[0]}_precision",
    "l2_precision": f"{le.classes_[1]}_precision",
    "l3_precision": f"{le.classes_[2]}_precision",
    "l1_recall": f"{le.classes_[0]}_recall",
    "l2_recall": f"{le.classes_[1]}_recall",
    "l3_recall": f"{le.classes_[2]}_recall",
})

df_results.to_csv(
    DATA_PATH + f"results_{col_target}_{model_checkpoint.replace('/', '_')}.csv", 
    index=False
)

col_probas = []
for j in range(len(test_dataset)):
  temp_probas = np.array([test_probas[i][j] for i in range(len(test_probas))])
  col_probas.append(temp_probas.sum(axis=0) / temp_probas.sum())

df_test[col_target] = le.inverse_transform(np.argmax(np.array(col_probas), axis=1))

df_probas = pd.DataFrame(col_probas).rename(
    columns={
        0: "prob_" + le.inverse_transform([0])[0],
        1: "prob_" + le.inverse_transform([1])[0],
        2: "prob_" + le.inverse_transform([2])[0],
    }
)

rows_prev = df_test.shape[0]
df_test = pd.concat([df_test, df_probas], axis=1)

if not use_full_dataset:
  df_test[["id", "link", "text", col_target] + df_probas.columns.tolist()].to_csv(
    DATA_PATH + f"sample_preds_{col_target}_{model_checkpoint.replace('/', '_')}.csv",
    index=False,
  )
else:
  df_test[["id_tweet", "link", "text", col_target] + df_probas.columns.tolist()].to_csv(
    DATA_PATH + f"output_{col_target}.csv",
    index=False,
  )

assert df_test.shape[0] == rows_prev

decisiones                            1115
agradecimientos_y_actos_simbolicos     556
estado_de_servicio                     287
Name: target_layer_2, dtype: int64
Training dccuchile/bert-base-spanish-wwm-cased
**** Split 0: 1762 observations for training / 196 observations for validation ****


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Epoch,Training Loss,Validation Loss,Kappa,F1,Precision,Recall,L1 F1,L2 F1,L3 F1,L1 Precision,L2 Precision,L3 Precision,L1 Recall,L2 Recall,L3 Recall
1,No log,1.044568,0.113924,0.338772,0.367816,0.377976,0.282051,0.734266,0.000000,0.500000,0.603448,0.000000,0.196429,0.937500,0.000000
2,1.091100,0.955201,0.291789,0.441940,0.429768,0.467262,0.582524,0.743295,0.000000,0.638298,0.651007,0.000000,0.535714,0.866071,0.000000
3,0.988300,0.812783,0.410326,0.609751,0.661667,0.586310,0.622642,0.752066,0.454545,0.660000,0.700000,0.625000,0.589286,0.812500,0.357143
4,0.812100,0.626935,0.552826,0.714246,0.699707,0.750000,0.679245,0.777778,0.685714,0.720000,0.807692,0.571429,0.642857,0.750000,0.857143
5,0.554500,0.555562,0.632500,0.766737,0.759453,0.794643,0.745098,0.819820,0.735294,0.826087,0.827273,0.625000,0.678571,0.812500,0.892857
6,0.389000,0.674448,0.616188,0.755587,0.781460,0.764881,0.681319,0.835443,0.750000,0.885714,0.792000,0.666667,0.553571,0.883929,0.857143


***** Running Evaluation *****
  Num examples = 196
  Batch size = 16
***** Running Evaluation *****
  Num examples = 196
  Batch size = 16
***** Running Evaluation *****
  Num examples = 196
  Batch size = 16
***** Running Evaluation *****
  Num examples = 196
  Batch size = 16
***** Running Evaluation *****
  Num examples = 196
  Batch size = 16
***** Running Evaluation *****
  Num examples = 196
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 196
  Batch size = 16


Saving model checkpoint to /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_0
Configuration saved in /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_0/config.json
Model weights saved in /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_0/pytorch_model.bin
***** Running Prediction *****
  Num examples = 403677
  Batch size = 16
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--dccuchile--bert-base-spanish-wwm-cased/snapshots/56a7647b957a4230fc3f80dafbe80f2ba9b0de73/config.json
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_no

...

**** Split 9: 1763 observations for training / 195 observations for validation ****


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--dccuchile--bert-base-spanish-wwm-cased/snapshots/56a7647b957a4230fc3f80dafbe80f2ba9b0de73/config.json
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LAB

Epoch,Training Loss,Validation Loss,Kappa,F1,Precision,Recall,L1 F1,L2 F1,L3 F1,L1 Precision,L2 Precision,L3 Precision,L1 Recall,L2 Recall,L3 Recall
1,No log,1.037408,0.187619,0.383937,0.355521,0.419913,0.465116,0.686695,0.000000,0.405405,0.661157,0.000000,0.545455,0.714286,0.000000
2,1.087300,0.916276,0.292079,0.440087,0.454762,0.461039,0.555556,0.764706,0.000000,0.714286,0.650000,0.000000,0.454545,0.928571,0.000000
3,0.975300,0.671027,0.530621,0.687865,0.729858,0.677381,0.704000,0.781818,0.577778,0.628571,0.796296,0.764706,0.800000,0.767857,0.464286
4,0.751400,0.496373,0.642186,0.768820,0.752765,0.793236,0.763636,0.824074,0.718750,0.763636,0.855769,0.638889,0.763636,0.794643,0.821429
5,0.500100,0.478163,0.643360,0.766643,0.777794,0.765909,0.755102,0.844828,0.700000,0.860465,0.816667,0.656250,0.672727,0.875000,0.750000
6,0.350600,0.486748,0.640458,0.773196,0.759457,0.799838,0.768000,0.805825,0.745763,0.685714,0.882979,0.709677,0.872727,0.741071,0.785714


***** Running Evaluation *****
  Num examples = 195
  Batch size = 16
***** Running Evaluation *****
  Num examples = 195
  Batch size = 16
***** Running Evaluation *****
  Num examples = 195
  Batch size = 16
***** Running Evaluation *****
  Num examples = 195
  Batch size = 16
***** Running Evaluation *****
  Num examples = 195
  Batch size = 16
***** Running Evaluation *****
  Num examples = 195
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 195
  Batch size = 16


Saving model checkpoint to /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_9
Configuration saved in /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_9/config.json
Model weights saved in /content/drive/MyDrive/nlp-tweets-classification/models/target_layer_2/beto_fold_9/pytorch_model.bin
***** Running Prediction *****
  Num examples = 403677
  Batch size = 16
